In [2]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import *
import numpy as np
import random
import torchvision.transforms as transforms
import torch.nn as nn
from tqdm import tqdm
import cv2
import timm

In [3]:
class CustomDataset(Dataset):
    def __init__(self, path, transform):
        self.path = path
        self.transform = transform                
        
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, idx):
        file = self.path[idx]
        
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        transformed = self.transform(image)
        
        return transformed

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.224,0.224,0.224))
])

In [4]:
import os

In [5]:
train_path = '/media/data2/rjsdn/dacon/basic_image_classification/train/'
test_path = '/media/data2/rjsdn/dacon/basic_image_classification/test/'

test = [test_path+x for x in sorted(os.listdir(test_path))]

train = ImageFolder(train_path,transform=transform)
test = CustomDataset(test,transform=transform)

ctoi = train.class_to_idx
itoc = {v: k for k, v in ctoi.items()}

In [6]:
testloader = DataLoader(test,batch_size=128,num_workers=4)
pred=[]

In [1]:
model1 = timm.create_model('efficientnet_b1',num_classes=10)
model1.load_state_dict(torch.load('weights/efficientnet_b1_224_124_1/best.path.tar')['model_state_dict'])
model2 = timm.create_model('efficientnet_b1',num_classes=10)
model2.load_state_dict(torch.load('weights/efficientnet_b1_224_124_2//best.path.tar')['model_state_dict'])
model3 = timm.create_model('efficientnet_b1',num_classes=10)
model3.load_state_dict(torch.load('weights/efficientnet_b1_224_124_3//best.path.tar')['model_state_dict'])

NameError: name 'timm' is not defined

In [8]:
model.cuda()
model.eval()
with torch.no_grad():
    for data in tqdm(testloader):
        x = data.cuda()
        output = model(x)
        output = torch.argmax(output,1)
        tmp = [itoc[x.item()] for x in output]
        pred=np.append(pred,tmp)
        


100%|██████████| 79/79 [00:11<00:00,  6.61it/s]


In [9]:
import pandas as pd

ss = pd.read_csv('/media/data2/rjsdn/dacon/basic_image_classification/sample_submission.csv')
ss.target=pred
ss.to_csv('sub.csv',index=False)